<p align="center">
    <img src="https://ars.els-cdn.com/content/image/3-s2.0-B9780081002858000055-f05-13-9780081002858.jpg" width="220" height="240" />

</p>

# Exploratory Data Analysis - Rod Pump Optimization

## Inventors Program Energy - NSC 325

#### Written by: Nicholas Khami, Nashra Ali, Vrishank Jannu

This notebook presents an implementation of a Random Forest Regression to predict the timing of a sucker rod pump failure. The data used for building the following model is provided by ConocoPhillips. The aim of this product is to assist production engineers in hypothesizing a better design for the downhole set-up in order to extend pump lifetimes. Thus, the company can cut yearly maintainence costs (i.e. part replacement) and consequently see an increase oil production per well. This benefits the company in guranteeing a higher return on their investments. 


### Imports

In [18]:
import glob
from datetime import datetime
import pandas as pd
import numpy as np
from scipy import stats
from scipy.integrate import quad
from scipy.optimize import curve_fit
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

### Minimal Data Cleaning
To begin approaching the problem, it is important to get a feel for the data that we will be working with. The following code displays the head of the data and dataframe properties. The data documents mechanical and chemical parameters of various sucker rod pumps owned by ConocoPhillips.

In [19]:
rpdf = pd.read_csv("../UT_RodPump_Dataset/rodpump_failure.csv")
rpdf.head()

,roduid,UWI,NODEID,IDWELL,tbguid,lifetime_start,lifetime_end,IDRECJOBPULL,REPORTTO,FAILSTART,FAILURETYPE,H2S_CONCENTRATION,PrimarySetpoint,SecondarySetpoint,StrokeLength,GrossStrokeLength,Fillage,YesterdaysAverageSPM,bha_configuration,chemgroup1_any,chemgroup1_all,chemgroup2_any,chemgroup2_all,chemgroup3_any,chemgroup3_all,max_unguided_dls,dls_high_in_hole,gas_anchor_length,MAX_INCLINATION,wellbore_category,manual_scale,packer_vs_tac,AVG_PRESS_FLOWLINE,AVG_PRESSURE_TUBING,AVG_PRESSURE_CASING,AVG_DIFFERENTIAL_PRESSURE,AVG_OIL_VOLUME,AVG_WATER_VOLUME,AVG_LIQUID_VOLUME,AVG_WATERSG,rod_sinker_type,rod_has_guides,rod_make,rod_apigrade,ROUTE,overall_max_sideload,shallow_max_sideload,max_unguided_sideload,DESANDDEGAS_TYP,CHROME_LENGTH,ENDURALLOY_LENGTH,POLY_LENGTH,NIPPLE_SET_DEPTH,pump_bore,gasanchor_od
0,GB27GKBE51029074693667,175-58-0111,box-child-doctor,GPVO63973435661154,GB55QFGB46756147811400,2014-05-02 00:00:00.000,2019-01-04 10:00:00.000,FGNB86581338411987,Tubing,2018-12-11 00:00:00.000,Tubing,0,75,60,144,108,91,6,TAC_ABOVE_NIP,1,1,1,1,0,0,2,1,19,2,Vertical,N,OTHER_ANCHOR,60,74,53,-21,51,59,109,1,SINKER_BARS_W_GUIDED_SUBS,False,strategize next-generation users,D,900,NaN,NaN,NaN,UNKNOWN,0,0,0,10025,1.5,NaN
1,GB53OEVX46438297645035,333-68-3523,outside-worry,ZAYT33358197650329,GB29QCCC14341267287129,2018-01-28 14:00:00.000,2019-05-17 12:00:00.000,XRYU81281518151403,Tubing,2019-04-01 00:00:00.000,Tubing,0,80,62,168,173,2,0,TAC_BELOW_NIP,1,1,0,0,0,0,2,2,17,2,Vertical,N,OTHER_ANCHOR,54,77,88,12,91,17,109,NaN,SINKER_BARS_W_GUIDED_SUBS,False,mesh enterprise portals,SpecialService,882,179,179,179,Miller LLC,0,0,0,10236,1.5,3.5
2,GB95BPWW35640301552066,165-51-5897,near-learn-simply,FNVL11432909873086,GB02DJAW44801752494129,2011-09-14 00:00:00.000,2012-08-20 00:00:00.000,PLPR44121073149707,Sucker Rod Pump,2012-08-01 00:00:00.000,Sucker Rod Pump,0,75,60,144,NaN,NaN,NaN,PACKER_DONNAN,0,0,0,0,0,0,5,5,21,2,Vertical,N,arrowset,NaN,74,56,-18,46,10,57,NaN,SLICK_SINKER_BARS,False,mesh enterprise portals,D,880,106,106,106,Miller LLC,0,0,0,10402,1.5,NaN
3,GB79UEDN31454825972680,543-56-3494,sea-improve-place,JOKL33317998159514,GB84GHPH97287631470412,2016-01-12 09:00:00.000,2017-03-16 16:30:00.000,MYCF04792093856141,Sucker Rod Pump,2017-01-19 00:00:00.000,Sucker Rod Pump,0,75,60,144,NaN,86,6,TAC_ABOVE_NIP,1,0,1,0,0,0,2,0,17,1,Vertical,Y,OTHER_ANCHOR,NaN,72,102,30,85,28,113,1,SINKER_BARS_W_GUIDED_SUBS,False,strategize next-generation users,D,875,98,48,98,Miller LLC,0,0,0,9558,1.75,4.5
4,GB20XCTM13691331349509,391-37-2039,personal-candidate,XOSU92041716672870,GB08VYTX61977431827206,2018-06-14 13:00:00.000,2020-04-15 17:37:11.338,WJDB83445325478746,NaN,NaN,NaN,0,80,65,168,127,61,4,TAC_BELOW_NIP,1,0,0,0,0,0,2,4,17,4,LowTangent,N,OTHER_ANCHOR,61,76,123,34,77,16,93,1,SUCKER_RODS_W_GUIDES,True,mesh enterprise portals,SpecialService,875,267,267,171,Miller LLC,0,129,0,9682,1.75,NaN


To begin cleaning, we convert unique well IDs to reformatted integer values and change lifetime_start, lifetime_end, and FAILSTART variables from date objects into integers. We create a new variable "lifetime" to represent the total time a particular pump operates (can be from the last failure time). This will serve as our response variable in the regression analysis. We will also at this stage drop rows with too many null values to be useful in training and testing and will replace any remaining null values with the integer value of 0.

In [15]:
rpdf = rpdf.dropna(subset=['FAILSTART'], axis=0)
rpdf['UWI'] = rpdf['UWI'].replace('-', '', regex=True).astype(int)

rpdf['lifetime_start'] = pd.to_datetime(rpdf['lifetime_start'], format="%Y-%m-%d %H:%M:%S.%f")
rpdf['lifetime_start'] = rpdf['lifetime_start'].astype(int)

rpdf['lifetime_end'] = pd.to_datetime(rpdf['lifetime_end'], format="%Y-%m-%d %H:%M:%S.%f")
rpdf['lifetime_end'] = rpdf['lifetime_end'].astype(int)

rpdf['FAILSTART'] = pd.to_datetime(rpdf['FAILSTART'], format="%Y-%m-%d %H:%M:%S.%f")
rpdf['FAILSTART'] = rpdf['FAILSTART'].astype(int)

rpdf['pump_bore'].replace(to_replace="Other", value=0, inplace=True)
rpdf['pump_bore'] = rpdf['pump_bore'].astype(float)

rpdf['lifetime'] = rpdf['lifetime_end'] - rpdf['lifetime_start']

TypeError: cannot astype a datetimelike from [datetime64[ns]] to [int32]

After doing some exploratory data analysis and reviewing summary statsitcs for each feauture, we found the following to be useful for our model: AVG_LIQUID_VOLUME (oil and water), overall_max_sideload, NIPPLE_SET_DEPTH, and all of the average pressures. There were several features such as StrokeLength and Fillage that we thought may be good predictors of failure, but they lacked differentiation throughout the dataframe. 

We think that the features listed are promising because they have differentiation across their quartiles and strong representation throughout the dataframe. Additionally, we hypothesize that they will reflect overall stress on the Rod Pump. 

In [20]:
rpdf.drop(['GrossStrokeLength', 'YesterdaysAverageSPM', 'shallow_max_sideload', 'max_unguided_sideload', 'CHROME_LENGTH', 'ENDURALLOY_LENGTH', 'POLY_LENGTH', 'gasanchor_od', 'chemgroup3_all', 'REPORTTO','H2S_CONCENTRATION','AVG_OIL_VOLUME','dls_high_in_hole'], axis=1, inplace=True)

Next, we assign an integer to each unique value for non-numeric columns of data (i.e. represented as integers or floats), known as label encoding. Any remaining null or incomprehensible (inf) values are replaced with the integer "0."

In [21]:
labelencoder = LabelEncoder()
rpdf_cleaned = rpdf.copy()

rpdf_cleaned['roduid'] = labelencoder.fit_transform(rpdf_cleaned['roduid'])
rpdf_cleaned['IDWELL'] = labelencoder.fit_transform(rpdf_cleaned['IDWELL'])
rpdf_cleaned['tbguid'] = labelencoder.fit_transform(rpdf_cleaned['tbguid'])
rpdf_cleaned['IDRECJOBPULL'] = labelencoder.fit_transform(rpdf_cleaned['IDRECJOBPULL'])
rpdf_cleaned['FAILURETYPE'] = labelencoder.fit_transform(rpdf_cleaned['FAILURETYPE'])
rpdf_cleaned['bha_configuration'] = labelencoder.fit_transform(rpdf_cleaned['bha_configuration'])
rpdf_cleaned['wellbore_category'] = labelencoder.fit_transform(rpdf_cleaned['wellbore_category'])
rpdf_cleaned['manual_scale'] = labelencoder.fit_transform(rpdf_cleaned['manual_scale'])
rpdf_cleaned['packer_vs_tac'] = labelencoder.fit_transform(rpdf_cleaned['packer_vs_tac'])
rpdf_cleaned['rod_has_guides'] = labelencoder.fit_transform(rpdf_cleaned['rod_has_guides'])
rpdf_cleaned['rod_make'] = labelencoder.fit_transform(rpdf_cleaned['rod_make'])
rpdf_cleaned['rod_apigrade'] = labelencoder.fit_transform(rpdf_cleaned['rod_apigrade'])
rpdf_cleaned['DESANDDEGAS_TYP'] = labelencoder.fit_transform(rpdf_cleaned['DESANDDEGAS_TYP'])
rpdf_cleaned['rod_sinker_type'] = labelencoder.fit_transform(rpdf_cleaned['rod_sinker_type'])
rpdf_cleaned['NODEID'] = labelencoder.fit_transform(rpdf_cleaned['NODEID'])

rpdf_cleaned.fillna(0, inplace=True)
rpdf_cleaned.replace([np.inf, -np.inf], 0, inplace=True)  

In [32]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.max_columns', 55)
rpdf_numeric_summary = rpdf_cleaned.describe()
print(rpdf_numeric_summary.shape)
rpdf_numeric_summary

(8, 38)


,roduid,NODEID,IDWELL,tbguid,IDRECJOBPULL,FAILURETYPE,PrimarySetpoint,SecondarySetpoint,StrokeLength,Fillage,bha_configuration,chemgroup1_any,chemgroup1_all,chemgroup2_any,chemgroup2_all,chemgroup3_any,max_unguided_dls,dls_high_in_hole,gas_anchor_length,MAX_INCLINATION,wellbore_category,manual_scale,packer_vs_tac,AVG_PRESS_FLOWLINE,AVG_PRESSURE_TUBING,AVG_PRESSURE_CASING,AVG_DIFFERENTIAL_PRESSURE,AVG_WATER_VOLUME,AVG_LIQUID_VOLUME,AVG_WATERSG,rod_sinker_type,rod_has_guides,rod_make,rod_apigrade,ROUTE,overall_max_sideload,DESANDDEGAS_TYP,NIPPLE_SET_DEPTH
count,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596,2596
mean,1298,309,316,911,1050,2,75,57,148,62,3,0,0,0,0,0,3,2,17,5,2,0,1,37,84,113,30,41,124,1,1,0,3,3,883,129,2,9758
std,750,183,186,527,492,1,7,16,15,42,1,0,0,0,0,0,6,6,7,5,1,0,1,31,64,107,125,48,95,0,1,0,1,1,12,141,2,1115
min,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-987,0,0,0,0,0,0,0,867,0,0,0
25%,649,152,150,461,633,1,75,60,144,0,1,0,0,0,0,0,1,1,17,2,1,0,0,0,62,64,-2,14,64,0,1,0,3,1,873,0,1,9601
50%,1298,306,316,913,1270,2,75,60,144,80,3,0,0,0,0,0,2,2,19,2,3,0,0,48,77,92,14,30,106,1,1,0,3,4,882,102,1,9915
75%,1946,465,475,1376,1457,3,75,65,146,93,4,1,0,0,0,0,3,3,21,6,3,0,2,62,100,138,49,52,159,1,1,1,4,4,886,203,4,10211
max,2595,631,632,1814,1692,3,91,100,336,655,5,1,1,1,1,1,92,92,45,53,4,1,5,248,1485,2322,4424,1067,1583,1,3,1,6,6,905,1351,5,12070


### Building the Model
We begin constructing our model by assigning a training and test set from the data. It is important that the training and test sets do not have any common data rows as this could affect the accuracy of our predictions. 70 percent of the cleaned data will be used for training while the remaining 30 percent will be used for testing.

In [9]:
X = rpdf_cleaned.drop(['FAILSTART', 'lifetime_start', 'lifetime_end', 'lifetime'], axis=1)
y = rpdf_cleaned['lifetime']

#split the data again into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

NameError: name 'rpdf_cleaned' is not defined

We now perform a grid search cross validation and randomized search cross validation to determine the best hyperparameters for the model. Based on score data, we will be using the hyperparameters determined by grid search CV for our random forest regression.

In [26]:
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}
rfr = RandomForestRegressor(random_state = 35)
g_search = GridSearchCV(estimator = rfr, param_grid = param_grid, cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)
g_search.fit(X_train, y_train)
print(g_search.best_params_)

NameError: name 'X_train' is not defined

In [1]:
g_search.best_score_

NameError: name 'g_search' is not defined

In [18]:
rfr_random_reg = RandomForestRegressor(random_state = 35)
n_estimators = [int(x) for x in np.linspace(start = 1, stop = 20, num = 20)] # number of trees in the random forest
max_features = ['auto', 'sqrt', 'log2'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points
r_grid = {'n_estimators': n_estimators, 'max_features': max_features, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}
print(r_grid)

{'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]}


In [19]:
rfr_random = RandomizedSearchCV(estimator=rfr_random_reg, param_distributions=r_grid, n_iter = 20, scoring='neg_mean_absolute_error', cv = 3, verbose=2, random_state=42, n_jobs=-1, return_train_score=True)
rfr_random.fit(X_train, y_train)
print(rfr_random.best_score_)
print(rfr_random.best_params_)

NameError: name 'X_train' is not defined

In [11]:
print(rfr_random.score(X_test , y_test))

NameError: name 'rfr_random' is not defined

We now create a random forest regressor object and pass our training set to it. The following code displays the weights of each predictor variable in relation to their correlation with sucker rod pump lifespan.

In [32]:
reg = RandomForestRegressor(bootstrap = True, max_depth = None, max_features = 'log2', n_estimators = 13)
reg.fit(X_train, y_train)
feature_names = list(X_train.columns)

importance = reg.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %s, Score: %.5f' % (feature_names[i],v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

NameError: name 'X_train' is not defined

The mean squared error (R^2) value gives us an idea of how accurate our model is in prediciting sucker rod pump lifetimes.

In [21]:
#score it up
reg.score(X_test, y_test)

NameError: name 'X_test' is not defined

We can finally visualize the results of our model through the following graphic.

In [31]:
# #and export so we can visualize what is going on
for k in range(12):
   dot_data = tree.export_graphviz(reg.estimators_[k+1], out_file='reggraph_' + str((k+1)), feature_names=X.columns, filled=True)
# tree.plot_tree(reg.estimators_[1])

NameError: name 'reg' is not defined